# MSDS 7349 Anonymizing Data
Authors: Alex Frye, Michael Smith, Lindsay Vitovsky

## Introduction
The Z-Virus has broken out in remote parts of the US. However, due to its incubation period, the virus was able to spread quickly before it was caught. The CDC in conjunction with the WHO have decided to realease sensitive healthcare data in an effort to crowdsource a solution to determine those attributes necessary to identify those immune to the disease and those that are carriers and their correlation with infected. With this data it may be possible to save the world by containing the diease before it spreads any further.

## Creating a Dataset - Michael
In order to illustrate the process of anonymizing and deanonymizing data, we needed a dataset that had a variety of features that ranged from categorical to quantitative data. As most of our research involved health care or demographically inclined data, we felt this was a good starting point, hence our appropriation of the Z-virus trend in pop culture.

Initially, we had originally sought out different datasets with the intention of either deanonymizing an already existing anonymous dataset, or anonymizing and then deanonymizing a publicly available dataset. Unfortunately, this led to two problems in attempting to use either option as an exercise. First, with an already anonymized dataset, we would have no way of verifying our results. Moreover, as we explain later in this example, properly anonymized data is virtually impossible to deanonymize. Second, there are some aspects of anonymization that are important to the process, but any publicly available datasets would have already gone through those processes (such as removal of PII). In order to make this example as complete as possible, it was necessary to build our own dataset from the ground up.

Creating a dataset isn't difficult. Most of us have some familiarity with either bootstrapping data or generating random bits and bytes. To that end, our first version of the dataset was remarkably simple. We randomly pulled from attribute lists, resulting in an even distribution of all feature values across all observations.

However...

This is a dataset that's meant to be anonymized and then deanonymized. This required thinking backwards, not from the idea of plaintext, but rather from the idea of what the resulting encrypted output might look like. And from there, understanding what techniques are necessary to conceal, hide, perturb, or manipulate data to prevent the ability of an attacker identifying a person or group of people. A set of completely random features and values with no correlated or causal relationships is incredibly effective against cryptanalysis attacks. While some datasets might truly be random, once our "version 1.0" dataset was anonymized, we would be unable to deanonymize it. But this led us to a realization about one of the weakness of anonymization in general: it's only as strong as how little the attacker knows about the dataset.

To that end, we relied heavily on demographic data to produce a somewhat statistically accurate selection of certain features such as ethnicity/race, hispanic/latino, age, blood type, hair color and eye color. If we were going to attempt to perform a logistic regression on the anonymized dataset, we would have also included controls for determining which features were selected for to produce our z-virus classifications, but we were primarily concerned with producing a dataset for which an attacker might know some information on what the dataset contained.

For our demographic data, we used the following sources:
* Demographic Data of Ethnicity and Age of USA: https://en.wikipedia.org/wiki/Demography_of_the_United_States  
* Hair and Eye Color data: http://www.gnxp.com/blog/2008/12/nlsy-blogging-eye-and-hair-color-of.php
* Blood Type: https://en.wikipedia.org/wiki/Blood_type_distribution_by_country

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
%%time

if os.path.isfile("dataset.csv"):
    print("Found the File!")
else:
    %run constructDataSet.py


In [ ]:
%%time
############################################################
# Load the Compiled Data from CSV
############################################################

# Create CSV Reader Function and assign column headers
def reader(f, columns):
    d = pd.read_csv(f)
    d.columns = columns
    return d


# Identify All CSV FileNames needing to be loaded
path = r''
all_files = glob.glob(os.path.join(path, "dataset.csv"))

# Define File Columns
columns = ["ID","LastName", "FirstName", "MiddleName", "Sex", "Age", "Ethnicity", "Hispanic_Latino", "BloodType", "HairColor", "EyeColor", "StreetAddress", "City", "State", "Zip", "PhoneNumber", "SocialSecurityNumber", "ZVirus"]

# Load Data
Data = pd.concat([reader(f, columns) for f in all_files])

#### The Final Dataset

Our final dataset consists of 18 features:

* **First Name** - Taken randomly from a list of popular given names
* **Middle Name** - Taken randomly from a list of popular given names
* **Last Name** - Taken randomly from a list of popular last names
* **Sex** - Biological sex, Binary value with a 50% chance of either/or based on US Census data
* **Age** - US census data divides up ages into groups, we randomly select for an age group based on that data and then randomly pick an age within that group range
* **Ethnicity** - Randomly selected based on US census data percentage of total population
* **Hispanic_Latino** - Binary option based on percentage of ethnicity that identifies as Hispanic/Latino from US Census data
* **Blood Type** - Based on blood type distribution
* **Hair Color** - Based on hair color distribution by ethnicity/race
* **Eye Color ** - Based on eye color distribution by ethnicity/race
* **Street Address** - Randomly generated 4 digit number from range 1000 - 9999 combined with list of 400 random street names
* **City** - Determined based on which state is randomly selected
* **State** - Randomly selected, all states are equally available to be selected for
* **Zip Code** - Based on which state is chosen, then random number is produced in range available to state.
* **Phone Number** - Area code is based on range controlled by state selection, combined with two randomly generated numbers in range 100-999 and 1000 - 9999
* **Social Security Number** - Three randomly generated numbers in ranges 100-999, 10-99, 1000-9999
* **Z-Virus** - each observation has a 50% chance to be immune, a 30% chance to be a carrier and a 20% chance to be infected

In [ ]:
display(Data.head())

## Anonymizing Data

The CDC in conjunction with the WHO want to release the dataset in order to crowdsource efforts to identify any underlying causes. However, there's the question of not only moral privacy concerns but those of legal ones as well. First and foremost this requires the identification and removal of Personally identifiable Information.

The following is identified as PII by the National Institute of Standards and Technology:

* **Names** - such as a person’s full name or an alias like an online handle or username
* **A Personal Identification Number** - a social security number, passport number, driver’s license number, or even a credit card number
* **Address** - usually understood as a full physical address, but could also be an email address
* **Asset Information** - this is information such as a user’s IP or MAC address, or any other host-specific, persistent identifiers like a browser fingerprint
* **Telephone Number** - taken in its entirety including area code; specified separate from personal identifcation number as individuals might share a single phone number
* **Personal Characteristics** - information like a photo, x-ray, fingerprints or other biometric data; this could also be unique features as well like scars, birthmarks, or physiological traits, including diseases.
* **Information Identifying Personal Property** -  such as a license plate, vehicle registration number, or other similar registrations; could also be unique characteristics similar to that of **Personal Characteristics**.
* **Linkable Information** - while some information might not be directly identifiable, with enough circumstantial and correlated information it would be possible to identify an individual

Immediately this means quite a few of our attributes should be removed for privacy and anonymity. Normally a last name by itself isn't considered personally identifiable, but we include several personal characteristics and distinguishing features as a part of our dataset. Because we want to maintain as much biological information as possible, We'll remove the First, Middle and Last name features all together.

Address, similarly is one of those bits of PII where each component on its own wouldn't be PII, but all together it's absolutely identifies an individual or small group of individuals. Most of what makes PII critical is that ability to reduce a group to an individual. Because geographic data might be important, we'll also remove most of the address data, except for the state, which will let any modelers maintain the ability to spot regional differences. Zip code could have also been used for greater specificity, but would depend on the spread of the virus.

Of course, phone number and social security number are right out, though area code could have been used in much the same way as zip code.

With all PII data removed, we're left with a limited amount of biological and geographical data for each observation. This data set already could probably be considered anonymized; with no truely unique bits of information, you're left with mostly nameless faces. But consider the data not in the context of this table, but in the real world. How many 12 year old, white non-hispanic males with type 0 positive blood, brown hair, and brown eyes are there in new mexico? Probably quite a few. But what about the rarer demographic attributes, such as native alaskan ethnicity or gray eyes? With only a 1% probably for either, it would be unlikely to come across too many of those in our dataset. Moreover, combinations of low probability features results in even more unique observations. This idea that linkable information can be considered personally identifiable leads us to the next step: K-Anonymization.

In [ ]:
#Output DataAnon as pd.frame

DataAnon = Data[["ID","Sex", "Age", "Ethnicity", "Hispanic_Latino", "BloodType", "HairColor", "EyeColor", "State", "ZVirus"]]
display(DataAnon.head())

#### K-Anonymization
The theory behind K-anonymization, suggests that by removing instances with less than K number of matching records, the data set seeks to protect the ease of finding loners by taking them out altogether. Attributes which may be used to easily identify an individual, and especially categorical data are key targets for K-anonymization techniques.

Attributes Age, Ethnicity, Blood Type, State, Hair Color, Eye Color, and Hispanic_Latino are all great candidates for examination with K-Anonymization, removing those elements that have a low repetition count.

The chosen "K" depends on the sample size of the original dataset, and contextual knowledge of the domain. We have chosen to set our "K" threshold for K-Anonymization to 5% of the original sample size. 
    <br> &emsp;&emsp;&emsp; $10000 \times .05 = 500$
    
This means, that for every attribute "class" we will analyze the frequency of that class, and for every class with less than 500 observations in the original dataset we will remove all observations in that class.  

In [ ]:
    # Define Threshold Value as 5% of Original Sample Size
KAThres = round(len(Data) * .05, 0)
print(KAThres)

#### Age
Age, given the large range of values, is not possible to apply K-Anonymization to each age by itself. Doing so, would remove all observations in our dataset and ignoring the attribute all-together would severely compromize the security of all individuals in our dataset. To mitigate this issue, we have chosen to split our Age attribute into 5 subcategories as defined below:
* <=20
* 21-40
* 41-60
* 61-80
* \>=81

Doing this allows us to continue sharing the Age attribute, without disregarding the security of our study participants. 

In [ ]:
%%time
%matplotlib inline


DataAnon["AgeClass"] = np.where(DataAnon["Age"] <= 20,                                                         "<=20",
                                np.where((DataAnon["Age"] >= 21) &  (DataAnon["Age"] <= 40),                   "21-40",
                                         np.where((DataAnon["Age"] >= 41) &  (DataAnon["Age"] <= 60),          "41-60",
                                                  np.where((DataAnon["Age"] >= 61) &  (DataAnon["Age"] <= 80), "61-80",
                                                                                                               ">=81"
                                                          )
                                                 )
                                        )
                               )

    #Agg Classes
AgeAgg = pd.DataFrame({'count' : DataAnon.groupby(["AgeClass"]).size()}).reset_index()

    # display class counts
display(AgeAgg)
    
    # Pie class Distribution
AgeAgg.plot.pie(y = 'count', labels = AgeAgg["AgeClass"], autopct='%1.1f%%', figsize=(12,6))


With our AgeClass category frequencies identified, we notice that the ">=81" demographic is very sparse, housing only a mere  364 individual (3.6%) of the dataset. These subjects would potentially be at risk for identification simply due to the sparcity of their occurance.

We have chosen to remove all observations for individuals identified in the >=81 demographic from the dataset. After removing records of Age Groups with a group population less than or equal to 500.0, we are left with 9635 records.

In [ ]:
    # MergeAgg
DataAnon = pd.merge(DataAnon, AgeAgg, on = "AgeClass")

    # Remove records with class count < Threshold Value
DataAnon = DataAnon[DataAnon["count"]>KAThres].drop(['Age', 'count'], axis = 1)

print("After removing records of Age Groups with a group population less than or equal to {0}, we are left with {1} records.".format(KAThres,len(DataAnon)))

    #Agg Classes
AgeAgg = pd.DataFrame({'count' : DataAnon.groupby(["AgeClass"]).size()}).reset_index()

    # display class counts
display(AgeAgg)

del AgeAgg

#### Ethnicity
With our Ethnicity frequencies identified, we notice that the we have three demographic categories that are very sparse. These subjects would potentially be at risk for identification simply due to the sparcity of their occurance.

We have chosen to remove Ethnicities: 
* "American Indian or Alaskan Native"
* "Asian American"
* "Native Hawaiian or Other Pacific Islander"

After removing records of Ethnicities with a group population less than or equal to 500.0, we are left with 8940 records.

In [ ]:
%%time
%matplotlib inline


    #Agg Classes
EthAgg = pd.DataFrame({'count' : DataAnon.groupby(["Ethnicity"]).size()}).reset_index()

    # display class counts
display(EthAgg)
    
    # Pie class Distribution
EthAgg.plot.pie(y = 'count', labels = EthAgg["Ethnicity"], autopct='%1.1f%%', figsize=(12,6))


In [ ]:
    # MergeAgg
DataAnon = pd.merge(DataAnon, EthAgg, on = "Ethnicity")

    # Remove records with class count < Threshold Value
DataAnon = DataAnon[DataAnon["count"]>KAThres].drop(['count'], axis = 1)

print("After removing records of ethnicities with a group population less than or equal to {0}, we are left with {1} records.".format(KAThres,len(DataAnon)))

    #Agg Classes
EthAgg = pd.DataFrame({'count' : DataAnon.groupby(["Ethnicity"]).size()}).reset_index()

    # display class counts
display(EthAgg)

del EthAgg

#### Blood Type
With our Blood Type frequencies identified, we notice that the we have three demographic categories that are very sparse. These subjects would potentially be at risk for identification simply due to the sparcity of their occurance.

We have chosen to remove Blood Types: 
* AB+
* AB-
* B-

After removing records of Blood Types with a group population less than or equal to 500.0, we are left with 8497 records.

In [ ]:
%%time
%matplotlib inline


    #Agg Classes
BTAgg = pd.DataFrame({'count' : DataAnon.groupby(["BloodType"]).size()}).reset_index()

    # display class counts
display(BTAgg)
    
    # Pie class Distribution
BTAgg.plot.pie(y = 'count', labels = BTAgg["BloodType"], autopct='%1.1f%%', figsize=(12,6))


In [ ]:
    # MergeAgg
DataAnon = pd.merge(DataAnon, BTAgg, on = "BloodType")

    # Remove records with class count < Threshold Value
DataAnon = DataAnon[DataAnon["count"]>KAThres].drop(['count'], axis = 1)

print("After removing records of Blood Types with a group population less than or equal to {0}, we are left with {1} records.".format(KAThres,len(DataAnon)))

    #Agg Classes
BTAgg = pd.DataFrame({'count' : DataAnon.groupby(["BloodType"]).size()}).reset_index()

    # display class counts
display(BTAgg)

del BTAgg

#### State
When identifying frequencies across states, we notice that every state is underneath our specified "K" threshold. However, we also notice that we have consistency across every state at approximately ~2% (+/- .4%) of the remaining dataset per state. Given the consistency of this attribute across the dataset, we have chosen to leave this data present in our anonymized dataset. We have chosen to do this because 1) The value of location may be extremely important to infection, and 2) Once perterbed later in the process, we feel this data will be more secure. 

In [ ]:
%%time
%matplotlib inline

    #Agg Classes
SAgg = pd.DataFrame({'count' : DataAnon.groupby(["State"]).size()}).reset_index()

    # display class counts
display(SAgg)
    
    # Pie class Distribution
SAgg.plot.pie(y = 'count', labels = SAgg["State"], autopct='%1.1f%%', figsize=(12,6))


#### Hair Color
With our Hair Color frequencies identified, we notice that the we have three demographic categories that are very sparse. These subjects would potentially be at risk for identification simply due to the sparcity of their occurance.

We have chosen to remove Blood Types: 
* Grey
* Light Blond
* Light Brown

After removing records of Hair Color with a group population less than or equal to 500.0, we are left with 8192 records.

In [ ]:
%%time
%matplotlib inline


    #Agg Classes
HairAgg = pd.DataFrame({'count' : DataAnon.groupby(["HairColor"]).size()}).reset_index()

    # display class counts
display(HairAgg)
    
    # Pie class Distribution
HairAgg.plot.pie(y = 'count', labels = HairAgg["HairColor"], autopct='%1.1f%%', figsize=(12,6))


In [ ]:
    # MergeAgg
DataAnon = pd.merge(DataAnon, HairAgg, on = "HairColor")

    # Remove records with class count < Threshold Value
DataAnon = DataAnon[DataAnon["count"]>KAThres].drop(['count'], axis = 1)

print("After removing records of Hair Color with a group population less than or equal to {0}, we are left with {1} records.".format(KAThres,len(DataAnon)))

    #Agg Classes
HairAgg = pd.DataFrame({'count' : DataAnon.groupby(["HairColor"]).size()}).reset_index()

    # display class counts
display(HairAgg)

del HairAgg

#### Eye Color
With our Eye Color frequencies identified, we notice that the we have five demographic categories that are very sparse. These subjects would potentially be at risk for identification simply due to the sparcity of their occurance.

We have chosen to remove Blood Types: 
* Black
* Grey
* Light Blue
* Light Brown
* Other

After removing records of Eye Color with a group population less than or equal to 500.0, we are left with 7662 records.

In [ ]:
%%time
%matplotlib inline


    #Agg Classes
EyeAgg = pd.DataFrame({'count' : DataAnon.groupby(["EyeColor"]).size()}).reset_index()

    # display class counts
display(EyeAgg)
    
    # Pie class Distribution
EyeAgg.plot.pie(y = 'count', labels = EyeAgg["EyeColor"], autopct='%1.1f%%', figsize=(12,6))


In [ ]:
    # MergeAgg
DataAnon = pd.merge(DataAnon, EyeAgg, on = "EyeColor")

    # Remove records with class count < Threshold Value
DataAnon = DataAnon[DataAnon["count"]>KAThres].drop(['count'], axis = 1)

print("After removing records of Eye Color with a group population less than or equal to {0}, we are left with {1} records.".format(KAThres,len(DataAnon)))

    #Agg Classes
EyeAgg = pd.DataFrame({'count' : DataAnon.groupby(["EyeColor"]).size()}).reset_index()

    # display class counts
display(EyeAgg)

del EyeAgg

#### Hispanic/Latino Population
With our Hispanic/Latino frequencies identified, we identify that both classes meet our "K" threshold. No observations are removed for this class.

In [ ]:
%%time
%matplotlib inline


    #Agg Classes
HisAgg = pd.DataFrame({'count' : DataAnon.groupby(["Hispanic_Latino"]).size()}).reset_index()

    # display class counts
display(HisAgg)
    
    # Pie class Distribution
HisAgg.plot.pie(y = 'count', labels = HisAgg["Hispanic_Latino"], autopct='%1.1f%%', figsize=(12,6))

#### Stratification of the Response Variable or Classifier

We are concerned about the distributions of each classifier matching publicly available data. For example, blood type distributions by ethnicity type are publicly available. The American Red Cross provides this information on their <a href="http://www.redcrossblood.org/learn-about-blood/blood-types">website</a>:

<table border="0" cellpadding="0" cellspacing="0" width="480">
<tbody>
<tr class="dates-bg-dkgray">
<td align="center" valign="top" width="40">
<div><b>&nbsp;</b></div>
</td>
<td align="center" valign="top" width="110">
<div><b>Caucasian</b></div>
</td>
<td align="center" valign="top" width="110">
<div><b>African- American</b></div>
</td>
<td align="center" valign="top" width="110">
<div><b>Latino-American</b></div>
</td>
<td align="center" valign="top" width="110">
<div><b>Asian</b></div>
</td>
</tr>
<tr class="dates-bg-gray">
<td align="center" valign="top">
<div><b>O +</b></div>
</td>
<td align="center" valign="top">
<div align="center">37%</div>
</td>
<td align="center" valign="top">
<div align="center">47%</div>
</td>
<td align="center" valign="top">
<div align="center">53%</div>
</td>
<td align="center" valign="top">
<div align="center">39%</div>
</td>
</tr>
<tr class="dates-bg-white">
<td align="center" valign="top">
<div><b>O -</b></div>
</td>
<td align="center" valign="top">
<div align="center">8%</div>
</td>
<td align="center" valign="top">
<div align="center">4%</div>
</td>
<td align="center" valign="top">
<div align="center">4%</div>
</td>
<td align="center" valign="top">
<div align="center">1%</div>
</td>
</tr>
<tr class="dates-bg-gray">
<td align="center" valign="top">
<div><b>A +</b></div>
</td>
<td align="center" valign="top">
<div align="center">33%</div>
</td>
<td align="center" valign="top">
<div align="center">24%</div>
</td>
<td align="center" valign="top">
<div align="center">29%</div>
</td>
<td align="center" valign="top">
<div align="center">27%</div>
</td>
</tr>
<tr class="dates-bg-white">
<td align="center" valign="top">
<div><b>A -</b></div>
</td>
<td align="center" valign="top">
<div align="center">7%</div>
</td>
<td align="center" valign="top">
<div align="center">2%</div>
</td>
<td align="center" valign="top">
<div align="center">2%</div>
</td>
<td align="center" valign="top">
<div align="center">0.5%</div>
</td>
</tr>
<tr class="dates-bg-gray">
<td align="center" valign="top">
<div><b>B +</b></div>
</td>
<td align="center" valign="top">
<div align="center">9%</div>
</td>
<td align="center" valign="top">
<div align="center">18%</div>
</td>
<td align="center" valign="top">
<div align="center">9%</div>
</td>
<td align="center" valign="top">
<div align="center">25%</div>
</td>
</tr>
<tr class="dates-bg-white">
<td align="center" valign="top">
<div><b>B -</b></div>
</td>
<td align="center" valign="top">
<div align="center">2%</div>
</td>
<td align="center" valign="top">
<div align="center">1%</div>
</td>
<td align="center" valign="top">
<div align="center">1%</div>
</td>
<td align="center" valign="top">
<div align="center">0.4%</div>
</td>
</tr>
<tr class="dates-bg-gray">
<td align="center" valign="top">
<div><b>AB +</b></div>
</td>
<td align="center" valign="top">
<div align="center">3%</div>
</td>
<td align="center" valign="top">
<div align="center">4%</div>
</td>
<td align="center" valign="top">
<div align="center">2%</div>
</td>
<td align="center" valign="top">
<div align="center">7%</div>
</td>
</tr>
<tr class="dates-bg-white">
<td align="center" valign="top">
<div><b>AB -</b></div>
</td>
<td align="center" valign="top">
<div align="center">1%</div>
</td>
<td align="center" valign="top">
<div align="center">0.3%</div>
</td>
<td align="center" valign="top">
<div align="center">0.2%</div>
</td>
<td align="center" valign="top">
<div align="center">0.1%</div>
</td>
</tr>
</tbody>
</table>

We fear, that information like this - and similar information for other attributes may make it possible to identify data even after pertubation (as discussed later). By pulling equal distributions of each ZVirus type, this slightly skews/changes the predictable distributions based on available population data.

When identifying how many records are remaining in each ZVirus category, we find that the least frequent ZVirus type is "Infected" with a frequency of 1528. In order to stratify, this means we cannot have a sample size larger than 1528 * 3 = 4584. 

In [ ]:
%%time
%matplotlib inline

ZVirusDist = pd.DataFrame({'count' : DataAnon.groupby(["ZVirus"]).size()}).reset_index()
display(ZVirusDist)

ZVirusDist.plot.pie(y = 'count', labels = ZVirusDist['ZVirus'], autopct='%1.1f%%')

del ZVirusDist

In order to truly randomly sample observations from each class and keep a fairly round sample size number, we have chosen to utilize a stratified sample size of 3750. This sample size will be stratified three ways in a 33/33/33 split (1250 Observations) across the ZVirus classes. 

We are able to compute the sample size for each ZVirus type, and then take a random sample within each group. Below you will see that our sampled distribution matches the chosen 33/33/33 split across ZVirus types. 

*Note:* A seed value equal to the sample size of each type in order to ensure reproducibility for this report.

In [ ]:
%%time
SampleSize = 3750

CarrierSample_Seed   = int(round(SampleSize * 33.3333 / 100.0,0))
ImmuneSample_Seed    = int(round(SampleSize * 33.3333 / 100.0,0))
InfectedSample_Seed  = int(round(SampleSize * 33.3333 / 100.0,0))


CarrierDataSampled  = DataAnon[DataAnon["ZVirus"] == 'Carrier'].sample(n=CarrierSample_Seed, replace = False, random_state = CarrierSample_Seed)
ImmuneDataSampled   = DataAnon[DataAnon["ZVirus"] == 'Immune'].sample(n=ImmuneSample_Seed, replace = False, random_state = ImmuneSample_Seed)
InfectedDataSampled = DataAnon[DataAnon["ZVirus"] == 'Infected'].sample(n=InfectedSample_Seed, replace = False, random_state = InfectedSample_Seed)


DataAnon = pd.concat([CarrierDataSampled,ImmuneDataSampled,InfectedDataSampled])

print(len(DataAnon))

ZVirusDist = pd.DataFrame({'count' : DataAnon.groupby(["ZVirus"]).size()}).reset_index()
display(ZVirusDist)

ZVirusDist.plot.pie(y = 'count', labels = ZVirusDist['ZVirus'], autopct='%1.1f%%')

del ZVirusDist

#### Perturbation

With our dataset stratified and properly K-Anonymized we're left with only high occurring observations distributed among equally sized classification groups. This is the bare minimum of what should ever be released. Now, some datasets will have slightly different requirements, but in order to reduce the probability of identifying an individual PII removal and K-Anonymization are necessary.

For some data sets, however, there's an additional step that can be taken to further secure the data: Perturbation.

In its simplest form (like below), the perturbation of data is simply running the table through a hashing algorithm to produce a one-way encrypted dataset. The best part about machine training/learning is that a computer has no cognitive understanding of the data. It simply identifies the string as a string and goes from there. While a random collection of characters and numbers might mean nothing to you, to the computer it will have value in its relation to the other strings of random characters. As such, our dataset containing entirely classificatory data can be run through a simple SHA256 hash (with a salt) to produce an encrypted dataset with no obvious way to decode the value of each feature for every observation.

For quantitative or linear features, a hashing algorithm won't work. For Age, one of our original quantitative features, we broke it up into classes and assigned each observation to a class. This is one way of anonymizing that data and did ultimately work with the method of hashing the data. Another way is to restric extreme values, by setting a cap on maximums and minimums. This requires sufficient knowledge of the dataset to identify local and global centers for the feature to best decide how to cap or control those extremes. Additionally, some fuzzing can be done to the values within a feature so no individual feature is accurate, but when taken on the aggregate the sample as whole still represents the population.

In [ ]:
#Saving off a copy of the data set after our KA
DataKAnon = DataAnon.copy()

In [ ]:
import hashlib

salt = "nintendo"

def hashbrowns(x):
    tohash = (salt + str(x)).encode('utf-8')
    return hashlib.sha256(tohash).hexdigest()

columns_to_hash = ["Sex","Ethnicity","Hispanic_Latino","BloodType","HairColor","EyeColor","State","AgeClass","ZVirus"]

for feature in columns_to_hash:
    DataAnon[feature] = DataAnon[feature].apply(hashbrowns)

display(DataAnon.head())

With the dataset properly anonymized we'll be exploring both the non-perturbed dataset and the perturbed dataset in much the same way an attacker might.

## Deanonymizing Data

An attacker, Bob, wants to identify individuals who participated in this study, to identify Blood Type and ZVirus Type information. Bob was informed on the date / time / location the texas study took place for data capture. Bob sat on a bench outside the facility, taking pictures of individuals as they walked in/out of the facility. Bob Logged timestamps from each photo identifying when individuals walked into the facility, and out of the facility so he could remove those individuals he perceived as employees due to long time durations. 

Since Texas is not the first state in which this study has taken place, Bob is familiar with the data shared by the CDC and their descriptions of the dataset values. Using pictures, he visually logged his perceived record value for the following attributes:
* Sex {0, 1} (i.e Female, Male)
* Ethnicity {Black American, White American, Other Race}
* Hispanic_Latino {0, 1} (i.e. No, Yes)
* HairColor(i.e. Black, Blond, Brown, Light Brown)
* State

To produce an example dataset of this visual attack, I have randomly selected 50 records from Texas in our original Dataset - storing only these attributes. In practice, there would be an error margin reducing the effectiveness of this data gathering technique, but for the purpose of this exercise we will select straight from the original data.

In [ ]:
%%time

BobData = Data[Data["State"] == "Texas"][["Sex", "Ethnicity", "Hispanic_Latino", "HairColor","State"]].sample(n=50, replace = False, random_state = 50)
display(BobData)
print(len(BobData))

#### Is Stratification and K-Anonymization after removing PII sufficient?

Pertubation on a dataset creates a number of problems when it comes to interpreting and utilizing the data, thus many times you may be inclined to avoid doing so. Given our dataset, shared after removing PII / K-Anon Records and Stratifying the result set, Bob is able to attempt to detect a given individual from his 50 observations. 

The first step in this process, is to aggregate the shared dataset on the attributes Bob was able to capture visually from his photographs ("Sex","Ethnicity", "Hispanic_Latino", "HairColor", and "State"), computing the count for each. This allows us to exploit the more unique characteristic of each observation by the combination of all values captured vs looking at one in particular. 

In [ ]:
%%time

    #Agg Classes
Agg = pd.DataFrame({'count' : DataKAnon.groupby(["Sex","Ethnicity", "Hispanic_Latino", "HairColor", "State"]).size()}).reset_index()

    # display class counts
display(Agg.head())
    


With our frequencies identified, we merge Bob's Data with the aggregated shared data, to find only those records from the shared dataset with matches on the demographic information captured from the photographs. Furthermore, for the purpose of identifying the highest likelihood match, we limit this merged data to those with a count of 1.

In [ ]:
    # MergeAgg
BobData = pd.merge(BobData, Agg, on = ["Sex", "Ethnicity", "Hispanic_Latino", "HairColor","State"])

display(BobData.sort_values(by=["count"]))

    # Remove records with class count < 1 Value
BobData = BobData[BobData["count"]==1].drop(['count'], axis = 1)

display(BobData)

This final result set below, is those individuals from the shared dataset you observed and have photos of that you have the highest confidence in a match. Further insights may be drawn with less confidence, but for the purpose of this report - only highest confidence matches have been printed here. Notice, I do not say that this result is with 100% confidence, even though these observations were classified as stand-alones in the shared dataset. Because we do not know if and what observations were removed from the original study, it is very possible that there was another observation in the study that was removed from the shared sample set we are unaware of. 

In [ ]:
    # Merge BobData with DataKAnon to identify specific individuals you observed
DataKAnonIdentified = pd.merge(BobData, DataKAnon, on = ["Sex", "Ethnicity", "Hispanic_Latino", "HairColor", "State"])
display(DataKAnonIdentified)

#### Perturbation

Give that Bob was ultimately able to identify two individuals even if he was using an associated dataset, perturbation certainly should be considered for all datasets and weighed against the difficulties of ultimately interpreting the final model. But is a perturbed dataset the end-all-be-all of anonymization? Could individuals still be identified?

Let's start over, but this time Bob has only a set of the perturbed data. If he can successfully reverse engineer the datset into human readable terms, then it's possible for him to identify an individual in the same method as above.

The first step, of course, is seeing if Bob can decode the hash/encryption.

#### Cracking the Hash

>A note on this process: We used a weak hashing method on purpose to illustrate a possible method of attack. A cryptographically secure method would not use a dictionary based salt, nor would that salt be static and independent of both the generator of the dataset and the recipient of the downloaded dataset.

Essentially, what we're attempting to do is create a forced hash collision by using what we know about the data. For example, both the Hispanic_Latino and the Sex columns appear to be binary however they have different values. This means context neutral classifier values such as 0 or 1 are unlikely for either one column or both. As such, it'll be simpler to start with the sex column as values within that feature would be limited to Male or Female, m or f, and other variations.

But before we even break out our coding IDE, we need to use a hacker's best friend: Google. All we have to do is copy and paste our given hashes into the search bar. This will let us know if the hashes have already been calculated before, and if they have, what the value is.

A quick search of either of the unique values in the Sex column tells us that either they aren't common values or a salt has been used (non exclusive or). If they were common values without a salt, then we would have turned up a few pages with the hash and probably its equivalency. For example, searching for "cd6357efdd966de8c0cb2f876cc89ec74ce35f0968e11743987084bd42fb8944" gives us "dog" using SHA256.

This means we have to attempt a bruteforce. Now we know based on the length of the hash, that it's most likely SHA256. What we don't know is how or if a salt was used. Normally in a better brute force function, there'd be a greater variety of attempts for mixing the salt with the hash function. There'd also likely be a larger dictionary and a few other tricks, but for the purposes of this exercise, the following is sufficient.

In [ ]:
%%time
f = open("10k_most_common.txt","r")
passwords = f.readlines()
f.close()

possibleValues = ['M','F','Male','Female','m','f','male','female','0','1','a','b','true','false','True','False','A','B']
valueTest = DataAnon["Hispanic_Latino"][0]

for password in passwords:
    password = password.strip('\n')
    for value in possibleValues:
        saltBefore = (password + str(value)).encode('utf-8')
        saltAfter = (str(value) + password).encode('utf-8')
        
        if (hashlib.sha256(saltBefore).hexdigest() == valueTest or hashlib.sha256(saltAfter).hexdigest() == valueTest):
            print("Password is: " + password)
        

If you'll noticed, we used an array that consisted of quite a few possible values that might fit in the column. For binary classifications, it's almost trivial how easy it might be to guess the values contained in the column. This means the security rests entirely on the salt. In this case, whoever chose the salt, chose poorly. And look at the time! Less than a second to evaluate from a list of 10,000 passwords. 

With the Password and Hash function now known, we can begin our attempts to guess the remaining data set. While hash functions are normally one way, as long as we know some basic meta information about the dataset it becomes possible to decode other attributes as it's much easier to guess the values of classifiers than it is to guess the salt for a hash.

In the case of ZVirus classification, we know that the CDC has been labeling their statistics and graphs with Carrier, Immune and Infected. So it's likely that these three values were used in this dataset especiall as it contains three unique classifiers in the ZVirus column. Of course it's possible they used non-contextual based classifiers such as 0, 1, 2, etc. Or even used partial contextual classifiers like C, Im, In. As always, start with the obvious, and work your way down.

In [ ]:
def hashdebrown(hashvalue,possibleValues):
    password = "nintendo" #taken from earlier
    for value in possibleValues:
        saltBefore = (password + str(value)).encode('utf-8')
        saltAfter = (str(value) + password).encode('utf-8')
        
        #sha256, again, taken from above
        if (hashlib.sha256(saltBefore).hexdigest() == hashvalue or hashlib.sha256(saltAfter).hexdigest() == hashvalue):
            return [hashvalue,value]
    return [hashvalue,"Unknown"]

#According to the CDC, they've been classifying people as Carrier, Immune or Infected
ZVirusValues = ["Carrier","Immune","Infected","0","1","2","C","Im","In", "A", "B"]
for hashvalue in DataAnon['ZVirus'].unique():
    print(str(hashdebrown(hashvalue,ZVirusValues)))

Each other feature follows the same path, taking educated guesses as to the values present, and running them against the given hashes. Once a pattern is identified it becomes even easier. For example, the hardest feature to guess is probably the Ethnicity/Race as those terms are somewhat  inconsistent across all reporters, but if you knew the CDC used the census classifications (as they should) then it becomes all the easier to crack.

But lets step back and assume that Bob couldn't crack the password and that whoever perturbed the data used a cryptographically strong method. Let's take a look at what he could do instead.

#### Metacontextual Analysis

Let's take another look at Bob's acquired dataset:

In [ ]:
DataMeta = DataAnon.copy()
DataMeta.head()

In particular I want to draw your attention to Eye Color and Hair Color.

In [ ]:
print(DataMeta["EyeColor"].unique())
print(DataMeta["HairColor"].unique())

Within this dataset, because all values for all features were applied the same salts, any features that use the same values will have the same hashes. In this case, "28e2cab6d46c9645d4026f6e7828a8bbe7205f4c15157f8fed7247fb4cb133a5" is repeated for both hair color and eye color. Going through a list of hair colors and eye colors, very few match up naturally on a person. Realistically speaking, the only color that appears in both hair and eyes (using common parlance) is brown in some fashion. Whether it's light brown, dark brown, or just brown, we can assume that "28e2cab6d46c9645d4026f6e7828a8bbe7205f4c15157f8fed7247fb4cb133a5" probably means brown in some way. This is an example of inferring meaning based on context and using any metaknowledge known about the dataset.

Let's take a look at Ethnicity next.

In [ ]:
EthnicityDist = pd.DataFrame({'Count' : DataMeta.groupby(["Ethnicity"]).size()}).reset_index()
display(EthnicityDist)

EthnicityDist.plot.pie(y = 'Count', labels = EthnicityDist['Ethnicity'], autopct='%1.1f%%')

Now, Bob, like us, has access to the US Census data and would know that there's not just three ethnicities or races. So what does this pie chart tell us? Well, either the data was prepared in such a way that ethnicities were grouped together or only the most common ethnicities were selected for representation. While it's possible that certain groups were combined, such as maybe Native Americans and other indigenous people, it's unlikely that, say, White Americans and Black Americans were grouped together. If Bob knows that White Americans make up around 73% of the population, Black Americans about 13%, and Asian Americans around 8%, those values are close enough that Bob could in fact assume that those hashes *equate in some way* to those demographic populations.

From here we can determine which value of Hispanic/Latino means "true" or "hispanic" by looking at the distribution of the White American Hispanic population.

In [ ]:
HispanicDist = pd.DataFrame({'Count' : DataMeta[DataMeta["Ethnicity"]=="41a04461d0ce7dfb47b0a33966f45f1ff9519d19ac4dddc84806843141131b08"].groupby(["Hispanic_Latino"]).size()}).reset_index()
display(HispanicDist)

HispanicDist.plot.pie(y = 'Count', labels = HispanicDist['Hispanic_Latino'], autopct='%1.1f%%')

And sure enough, since Bob knows that there are far more Non-Hispanic White Americans than Hispanic White Americans, it's easy to see which value translates into "True" or "Hispanic". So lets go through and replace the values we've been able to infer within the dataset and take a look at it in comparison to our original Kanonymized data.

In [ ]:
DataMeta = DataMeta.replace("41a04461d0ce7dfb47b0a33966f45f1ff9519d19ac4dddc84806843141131b08","White")
DataMeta = DataMeta.replace("239bfd6eacbc79744e8c47eed8c3b729a6144861a58955b5372c3bfb1eefb5aa","Black")
DataMeta = DataMeta.replace("f2c2851f19dca29cb926e9bd781a2ff31214d0254386e9f634ddf175c84530eb","Asian")
DataMeta = DataMeta.replace("28e2cab6d46c9645d4026f6e7828a8bbe7205f4c15157f8fed7247fb4cb133a5","Brown")
DataMeta = DataMeta.replace("ddebf4bb08617e33fac3c0e43ea5c3f63df912887f984d80d61d4b685a036dc3","Non-Hispanic")
DataMeta = DataMeta.replace("aec22b5de39a4ec587c05b0bedac29de567dfa442608cadcaa3afa497c263d3b","Hispanic")

display(DataMeta.head())
display(DataKAnon.head())

Not every feature will be able to be inferred in this fashion. Bloodtype will (based on how we built our dataset), but for features like Sex or State, simple probability or statistical demographic analysis won't be enough. However, much like the popular number game Sudoku, once you start filling in the easy blanks, the harder blanks will follow. For example, we can verify our Ethnicity selections by looking at Hair and Eye color demographics and comparing them to our assumptions. Further, this is based entirely on this dataset alone. If Bob has access to any additional data, it just gets easier and easier to make those metacontextual connections necessary to identify feature values.

#### Stopping Metacontextual Analysis

Interestingly, after playing with the data, Age Class appeared to be the most difficult to predict and crack. Because the groupings didn't follow census data, but rather an internal logic, it would have been almost impossible for bob to put together which class/group equated to which hash, or even how to group ages together to begin with.

Certainly census data would be easy enough to deanonymize, but the only reason it worked so well with this dataset is because our original dataset proportionally represented a real population. Most datasets of this nature probably won't actually represent a real population, but it's important to remember for those datasets that do, especially for an extremely large number of observations. And in actuality, the easiest data to deanonymize were for features that had easily predictable values. 

## Conclusion (needs cleaning up.. just alex thoughts)

**K-Anonymization**

* When removing so many observations / entire demographic categories, reduces the effectiveness / usefullness of results provided back from open source contributors. Any algorithms produced, identifying ways to predict Carriers vs. Infected, etc. would be subject ONLY to the demographic provided to them and should not be used on any demographic removed from this dataset. This means, that those outlier demographics in the data would not benefit from this study in the same way others would.
* Sample size is important!!!! Given our relatively small sample size after our anonymization techniques, it became relatively easy to identify an individual. With a larger sample size, the number of duplicates with the same demographic characteristics would increase (making outliers less of targets), fewer classes would have needed to be removed, and ultimately results provided back to us would be more meaningful. One way we could have accomplished this, would be to perform a bootstrap on our original dataset with replacement to produce a much larger original dataset to work with. 
* Combinations of attributes is just as important, if not more important, than the attribute itself. Although we eliminated outlier classes through utilization of our "K" threshold, we did not analyze the frequency of combinations of attributes. This left a huge security gap in our dataset that was very easy for our attacker, Bob, to exploit. Also, leaving in State was a mistake during K-Anonymization, as the number of observations remaining once narrowed down to a state was VERY small. This was a key component to Bob's attack.